<a href="https://colab.research.google.com/github/chongzicbo/Dive-into-Deep-Learning-tf.keras/blob/master/5.10.%20%E6%89%B9%E9%87%8F%E5%BD%92%E4%B8%80%E5%8C%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##5.10. 批量归一化
本节我们介绍批量归一化（batch normalization）层，它能让较深的神经网络的训练变得更加容易 。标准化处理输入数据使各个特征的分布相近：这往往更容易训练出有效的模型。

通常来说，数据标准化预处理对于浅层模型就足够有效了。随着模型训练的进行，当每层中参数更新时，靠近输出层的输出较难出现剧烈变化。但对深层神经网络来说，即使输入数据已做标准化，训练中模型参数的更新依然很容易造成靠近输出层输出的剧烈变化。这种计算数值的不稳定性通常令我们难以训练出有效的深度模型。

批量归一化的提出正是为了应对深度模型训练的挑战。在模型训练时，批量归一化利用小批量上的均值和标准差，不断调整神经网络中间输出，从而使整个神经网络在各层的中间输出的数值更稳定。批量归一化和下一节将要介绍的残差网络为训练和设计深度模型提供了两类重要思路。

###5.10.1. 批量归一化层
对全连接层和卷积层做批量归一化的方法稍有不同。下面我们将分别介绍这两种情况下的批量归一化。

####5.10.1.1. 对全连接层做批量归一化
我们先考虑如何对全连接层做批量归一化。通常，我们将批量归一化层置于全连接层中的仿射变换和激活函数之间。设全连接层的输入为 $u$ ，权重参数和偏差参数分别为 $W$ 和 $b$ ，激活函数为 $ϕ$ 。设批量归一化的运算符为 $BN$ 。那么，使用批量归一化的全连接层的输出为
$$\phi(\text{BN}(\boldsymbol{x})),$$
其中批量归一化输入 $x$ 由仿射变换
$$\boldsymbol{x} = \boldsymbol{W\boldsymbol{u} + \boldsymbol{b}}$$
得到。考虑一个由 m 个样本组成的小批量，仿射变换的输出为一个新的小批量$\mathcal{B} = \{\boldsymbol{x}^{(1)}, \ldots, \boldsymbol{x}^{(m)} \}$。它们正是批量归一化层的输入。对于小批量 $B$ 中任意样本$\boldsymbol{x}^{(i)} \in \mathbb{R}^d, 1 \leq i \leq m$，批量归一化层的输出同样是d维向量
$$\boldsymbol{y}^{(i)} = \text{BN}(\boldsymbol{x}^{(i)}),$$
并由以下几步求得。首先，对小批量$ B$ 求均值和方差：
$$\boldsymbol{\mu}_\mathcal{B} \leftarrow \frac{1}{m}\sum_{i = 1}^{m} \boldsymbol{x}^{(i)},$$
$$\boldsymbol{\sigma}_\mathcal{B}^2 \leftarrow \frac{1}{m} \sum_{i=1}^{m}(\boldsymbol{x}^{(i)} - \boldsymbol{\mu}_\mathcal{B})^2,$$
其中的平方计算是按元素求平方。接下来，使用按元素开方和按元素除法对 $x^{(i)}$ 标准化：
$$\hat{\boldsymbol{x}}^{(i)} \leftarrow \frac{\boldsymbol{x}^{(i)} - \boldsymbol{\mu}_\mathcal{B}}{\sqrt{\boldsymbol{\sigma}_\mathcal{B}^2 + \epsilon}},$$

这里 $\epsilon>0$ 是一个很小的常数，保证分母大于0。在上面标准化的基础上，批量归一化层引入了两个可以学习的模型参数，拉伸（scale）参数  $\gamma$  和偏移（shift）参数  $\beta$。这两个参数和 $x^{(i)}$ 形状相同，皆为 d 维向量。它们与 $\hat{x}^{(i)}$ 分别做按元素乘法（符号 ⊙ ）和加法计算：
$${\boldsymbol{y}}^{(i)} \leftarrow \boldsymbol{\gamma} \odot \hat{\boldsymbol{x}}^{(i)} + \boldsymbol{\beta}.$$

至此，我们得到了 $x^{(i)}$ 的批量归一化的输出 $y^{(i)}$ 。 值得注意的是，可学习的拉伸和偏移参数保留了不对 $x^{(i)}$ 做批量归一化的可能：此时只需学出 $\boldsymbol{\gamma} = \sqrt{\boldsymbol{\sigma}_\mathcal{B}^2 + \epsilon}$和$\boldsymbol{\beta} = \boldsymbol{\mu}_\mathcal{B}$。我们可以对此这样理解：如果批量归一化无益，理论上，学出的模型可以不使用批量归一化。

####5.10.1.2. 对卷积层做批量归一化
对卷积层来说，批量归一化发生在卷积计算之后、应用激活函数之前。如果卷积计算输出多个通道，我们需要对这些通道的输出分别做批量归一化，且每个通道都拥有独立的拉伸和偏移参数，并均为标量。设小批量中有 $m$ 个样本。在单个通道上，假设卷积计算输出的高和宽分别为 $p$ 和 $q$ 。我们需要对该通道中 $m \times p \times q$ 个元素同时做批量归一化。对这些元素做标准化计算时，我们使用相同的均值和方差，即该通道中 $m \times p \times q$ 个元素的均值和方差。

####5.10.1.3. 预测时的批量归一化
使用批量归一化训练时，我们可以将批量大小设得大一点，从而使批量内样本的均值和方差的计算都较为准确。将训练好的模型用于预测时，我们希望模型对于任意输入都有确定的输出。因此，单个样本的输出不应取决于批量归一化所需要的随机小批量中的均值和方差。一种常用的方法是通过移动平均估算整个训练数据集的样本均值和方差，并在预测时使用它们得到确定的输出。可见，和丢弃层一样，批量归一化层在训练模式和预测模式下的计算结果也是不一样的。

###5.10.2. 从零开始实现
下面我们来实现批量归一化层。

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Sequential
from tensorflow.data import Dataset
from tensorflow.keras import losses,optimizers
import time
import numpy as np
from tensorflow import image
import os
import sys

In [0]:
tf.enable_eager_execution() #启用动态图计算，tf2.x不用调用该函数

In [0]:
def batch_norm(X,gamma,beta,moving_mean,moving_var,eps,momentum,is_training=True):
  if not is_training:
    #如果是预测模式下，直接使用传入的移动平均所得的均值和方差
    X_hat=(X-moving_mean)/tf.sqrt(moving_var+eps)
  else:
    assert len(X.shape) in (2,4)

    if len(X.shape)==2:
      #使用全连接层的情况，计算特征维上的均值和方差
      mean=tf.reduce_mean(X,axis=0)
      var=tf.reduce_mean(tf.square(X-mean),axis=0)
    else:
      #使用二维卷积层的情况，计算通道维上(axis=-1)的均值和方差，这里我们需要保持X的形状以便后面可以做广播运算
      mean=tf.reduce_mean(X,axis=(0,1,2),keepdims=True)
      var=tf.reduce_mean(tf.square((X-mean)),axis=(0,1,2),keepdims=True)
    #训练模式下调用当前的均值和方差做标准化
    X_hat=(X-mean)/tf.sqrt(var+eps)
    #更新移动平均的均值和方差
    moving_mean=momentum*moving_mean+(1.0-momentum)*mean
    moving_var=momentum*moving_var+(1.0-momentum)*var
  Y=gamma*X_hat+beta #拉伸和偏移
  return Y,moving_mean,moving_var

接下来，我们自定义一个BatchNorm层。它保存参与求梯度和迭代的拉伸参数gamma和偏移参数beta，同时也维护移动平均得到的均值和方差，以便能够在模型预测时被使用。BatchNorm实例所需指定的num_features参数对于全连接层来说应为输出个数，对于卷积层来说则为输出通道数。该实例所需指定的num_dims参数对于全连接层和卷积层来说分别为2和4。

In [0]:
class BatchNorm(layers.Layer):
  def __init__(self,num_features,num_dims):
    super(BatchNorm,self).__init__()
    if num_dims==2:
      shape=(1,num_features)
    else:
      shape=(1,1,1,num_features)
    #参与求梯度和迭代的拉伸和偏移参数，分布初始化为1和0
    self.gamma=tf.Variable(tf.constant(1,shape=shape,dtype=tf.float32))
    self.beta=tf.Variable(tf.zeros(shape=shape,dtype=tf.float32))

    #不参与求梯度和迭代的变量，初始化成0
    self.moving_mean=tf.zeros(shape=shape)
    self.moving_var=tf.zeros(shape=shape)

  def call(self,X):
    Y,self.moving_mean,self.moving_var=batch_norm(X,self.gamma,self.beta,self.moving_mean,self.moving_var,eps=1e-5,momentum=0.8)
    return Y

####5.10.2.1. 使用批量归一化层的LeNet
下面我们修改“卷积神经网络（LeNet）”这一节介绍的LeNet模型，从而应用批量归一化层。我们在所有的卷积层或全连接层之后、激活层之前加入批量归一化层。

In [0]:
net=Sequential([
  layers.Conv2D(6,kernel_size=4),
  BatchNorm(6,num_dims=4),
  layers.Activation('sigmoid'),
  layers.MaxPool2D(pool_size=2,strides=2),
  layers.Conv2D(16,kernel_size=5),
  BatchNorm(16,num_dims=4),
  layers.Activation('sigmoid'),
  layers.MaxPool2D(pool_size=2,strides=2), 
  layers.Flatten(),
  layers.Dense(120),
  BatchNorm(120,num_dims=2),
  layers.Activation('sigmoid'),
  layers.Dense(84),
  BatchNorm(84,num_dims=2),
  layers.Activation('sigmoid'),
  layers.Dense(10)
]
)

In [0]:
X=tf.random.uniform(shape=(1,224,224,3))
for layer in net.layers:
  # print(type(layer))
  # print(layer.name)
  X=layer(X)
  print(layer.name,'output shape:\t',X.shape)

conv2d
conv2d output shape:	 (1, 221, 221, 6)
batch_norm
(1, 221, 221, 6)
batch_norm output shape:	 (1, 221, 221, 6)
activation
activation output shape:	 (1, 221, 221, 6)
max_pooling2d
max_pooling2d output shape:	 (1, 110, 110, 6)
conv2d_1
conv2d_1 output shape:	 (1, 106, 106, 16)
batch_norm_1
(1, 106, 106, 16)
batch_norm_1 output shape:	 (1, 106, 106, 16)
activation_1
activation_1 output shape:	 (1, 106, 106, 16)
max_pooling2d_1
max_pooling2d_1 output shape:	 (1, 53, 53, 16)
flatten
flatten output shape:	 (1, 44944)
dense
dense output shape:	 (1, 120)
batch_norm_2
(1, 120)
batch_norm_2 output shape:	 (1, 120)
activation_2
activation_2 output shape:	 (1, 120)
dense_1
dense_1 output shape:	 (1, 84)
batch_norm_3
(1, 84)
batch_norm_3 output shape:	 (1, 84)
activation_3
activation_3 output shape:	 (1, 84)
dense_2
dense_2 output shape:	 (1, 10)


In [0]:
y=net(tf.random.uniform(shape=(1,224,224,3)))

(1, 221, 221, 6)
(1, 106, 106, 16)
(1, 120)
(1, 84)


下面我们训练修改后的模型。

In [0]:
lr,num_epochs,batch_size=0.05,5,128
optimizer=optimizers.SGD(learning_rate=lr)
loss=losses.SparseCategoricalCrossentropy(from_logits=True)
buffer_size=1000
def load_data_fashion_mnist(batch_size,buffer_size):
  (x_train,y_train),(x_test,y_test)=keras.datasets.fashion_mnist.load_data()
  x_train=x_train[:,:,:,np.newaxis]#将三维张量增加一个channel维
  x_test=x_test[:,:,:,np.newaxis]
  train_iter=Dataset.from_tensor_slices((x_train,y_train)).map(lambda x,y:(x/255,y)).shuffle(buffer_size).batch(batch_size).map(lambda x,y:(image.resize(images=x,size=(224,224)),y))
  test_iter=Dataset.from_tensor_slices((x_test,y_test)).map(lambda x,y:(x/255,y)).batch(batch_size).map(lambda x,y:(image.resize(images=x,size=(224,224)),y))
  return train_iter,test_iter
train_iter,test_iter=load_data_fashion_mnist(batch_size=batch_size,buffer_size=buffer_size)

In [0]:
net.compile(optimizer=optimizer,loss=loss,metrics=['accuracy'])
history=net.fit_generator(train_iter,validation_data=test_iter,epochs=num_epochs)

Epoch 1/5
469/469 [==============================] - 170s 362ms/step - loss: 0.6893 - acc: 0.7785 - val_loss: 0.4985 - val_acc: 0.8527
Epoch 2/5
469/469 [==============================] - 163s 349ms/step - loss: 0.4315 - acc: 0.8640 - val_loss: 0.4048 - val_acc: 0.8715
Epoch 3/5
469/469 [==============================] - 164s 349ms/step - loss: 0.3536 - acc: 0.8830 - val_loss: 0.3602 - val_acc: 0.8795
Epoch 4/5
469/469 [==============================] - 163s 349ms/step - loss: 0.3088 - acc: 0.8966 - val_loss: 0.3359 - val_acc: 0.8849
Epoch 5/5
469/469 [==============================] - 164s 349ms/step - loss: 0.2746 - acc: 0.9067 - val_loss: 0.3152 - val_acc: 0.8912


###5.10.3. 小结
* 在模型训练时，批量归一化利用小批量上的均值和标准差，不断调整神经网络的中间输出，从而使整个神经网络在各层的中间输出的数值更稳定。
* 对全连接层和卷积层做批量归一化的方法稍有不同。
* 批量归一化层和丢弃层一样，在训练模式和预测模式的计算结果是不一样的。